In [1]:
# run this after
#     ./1_populate_tables.ipynb

In [2]:
from numpy import std, mean
import sqlite3
db_fname = '../../hmt.db'

features = [
    'pitch_min', 'pitch_max', 'pitch_mean', 'pitch_std', 
    'rate_syl', 'rate_vcd',
    'intensity_min', 'intensity_max', 'intensity_mean', 'intensity_std', 
    'jitter', 'shimmer', 'nhr'
]

In [3]:
print('normalizing; stats before update: ', end='\n\n')
with sqlite3.connect(db_fname) as conn:
    c = conn.cursor()
    
    c.execute('SELECT spk_id FROM speakers ORDER BY spk_id;')
    spk_ids = [v[0] for v in c.fetchall()]
    
    for spk_id in spk_ids:
        print('spk_id: %d' % spk_id)
        for f in features:
            # chunks with non-null values for current feature and speaker
            sql_fragment = \
                'FROM   chunks chu ' \
                'JOIN   turns tur ' \
                'ON     chu.tur_id == tur.tur_id ' \
                'JOIN   tasks tsk ' \
                'ON     tur.tsk_id == tsk.tsk_id ' \
                'JOIN   sessions ses ' \
                'ON     tsk.ses_id == ses.ses_id ' \
                'JOIN   speakers spk_a ' \
                'ON     ses.spk_id_a == spk_a.spk_id ' \
                'JOIN   speakers spk_b ' \
                'ON     ses.spk_id_b == spk_b.spk_id ' \
                'WHERE  CASE ' \
                '           WHEN tsk.a_or_b == "A" AND ' \
                '                tur.speaker_role == "d" ' \
                '           THEN spk_a.spk_id ' \
                '           WHEN tsk.a_or_b == "B" AND ' \
                '                tur.speaker_role == "f" ' \
                '           THEN spk_a.spk_id ' \
                '           ELSE spk_b.spk_id ' \
                '       END == ' + str(spk_id) + ' ' \
                'AND    ' + f + ' IS NOT NULL'

            # non-null feature values for current feature and speaker
            sql_stmt1 = \
                'SELECT ' + f + ' ' + sql_fragment
                
            # normalization for current feature and speaker
            # using given means, std
            sql_stmt2 = \
                'UPDATE chunks SET ' + ('%s = (%s-?)/?' % (f, f)) + ' ' \
                'WHERE  chu_id IN (' \
                    'SELECT chu.chu_id ' + sql_fragment + ')'
            
            c.execute(sql_stmt1,)
            
            data = [v[0] for v in c.fetchall()]
            stats = (mean(data), std(data) if std(data) != 0.0 else 1.0) 
            
            print('\t%s, mean: %.3f, std: %.3f' % (f, stats[0], stats[1]))
            c.execute(sql_stmt2, (stats[0], stats[1]))
        conn.commit()
        print('\n\n')

normalizing; stats before update: 

spk_id: 1
	pitch_min, mean: 106.017, std: 15.098
	pitch_max, mean: 271.890, std: 172.264
	pitch_mean, mean: 139.345, std: 22.275
	pitch_std, mean: 36.179, std: 41.972
	rate_syl, mean: 5.801, std: 2.588
	rate_vcd, mean: 0.621, std: 0.217
	intensity_min, mean: 43.295, std: 7.651
	intensity_max, mean: 65.518, std: 4.989
	intensity_mean, mean: 57.303, std: 4.555
	intensity_std, mean: 5.943, std: 2.486
	jitter, mean: 0.013, std: 0.009
	shimmer, mean: 0.110, std: 0.039
	nhr, mean: 0.159, std: 0.097



spk_id: 2
	pitch_min, mean: 93.369, std: 33.741
	pitch_max, mean: 304.558, std: 180.714
	pitch_mean, mean: 130.151, std: 46.222
	pitch_std, mean: 44.991, std: 42.817
	rate_syl, mean: 5.904, std: 2.620
	rate_vcd, mean: 0.479, std: 0.226
	intensity_min, mean: 46.111, std: 7.183
	intensity_max, mean: 74.245, std: 5.105
	intensity_mean, mean: 64.513, std: 3.962
	intensity_std, mean: 7.189, std: 2.429
	jitter, mean: 0.020, std: 0.014
	shimmer, mean: 0.110, std: 0.

	rate_syl, mean: 5.397, std: 1.809
	rate_vcd, mean: 0.660, std: 0.151
	intensity_min, mean: 40.278, std: 8.136
	intensity_max, mean: 69.269, std: 3.787
	intensity_mean, mean: 60.135, std: 3.446
	intensity_std, mean: 7.268, std: 2.353
	jitter, mean: 0.012, std: 0.006
	shimmer, mean: 0.090, std: 0.031
	nhr, mean: 0.119, std: 0.071



spk_id: 113
	pitch_min, mean: 144.015, std: 51.814
	pitch_max, mean: 382.560, std: 133.762
	pitch_mean, mean: 213.367, std: 29.486
	pitch_std, mean: 50.135, std: 30.162
	rate_syl, mean: 5.059, std: 2.064
	rate_vcd, mean: 0.679, std: 0.178
	intensity_min, mean: 42.105, std: 8.221
	intensity_max, mean: 69.914, std: 4.404
	intensity_mean, mean: 60.262, std: 3.911
	intensity_std, mean: 7.367, std: 2.727
	jitter, mean: 0.010, std: 0.004
	shimmer, mean: 0.095, std: 0.030
	nhr, mean: 0.122, std: 0.080



spk_id: 114
	pitch_min, mean: 106.617, std: 19.111
	pitch_max, mean: 235.406, std: 105.691
	pitch_mean, mean: 150.368, std: 19.551
	pitch_std, mean: 27.019, std: 1

In [4]:
# run script to create auxiliary tables (involves randomness!)
with open('../sql/aux_tables.sql') as sql_file:
    sql_script = ''.join(sql_file.readlines())
    with sqlite3.connect(db_fname) as conn:
        conn.cursor().executescript(sql_script)